In [1]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [2]:
dir=r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\Spotify_Million\data'

In [3]:
# Initialize data structures for deduplication
artist = {}
tracks = {}
album = {}

# Helper function to get or create IDs for deduplication
def get_or_create_id(mapping, value):
    if value not in mapping:
        mapping[value] = len(mapping) + 1
    return mapping[value]

# Directory containing JSON files

# Initialize an empty list for storing playlists as rows
playlists_data = []

for filename in tqdm(os.listdir(dir)):
    if filename.endswith(".json"):
        dir_file = os.path.join(dir, filename)
        try:
            with open(dir_file, "r") as json_file:
                data = json.load(json_file)

                for playlist in data.get("playlists", []):
                    playlist_tracks = []

                    for track in playlist.get("tracks", []):
                        playlist_tracks.append([get_or_create_id(artist, (track.get("artist_uri"))), get_or_create_id(album, (track.get("album_uri"))), get_or_create_id(tracks, (track.get("track_uri"))), track.get("duration_ms")//1000])

                    # Add the playlist to the dataframe structure
                    playlists_data.append({
                        "user_id": playlist.get("pid"),
                        "tracks": playlist_tracks
                    })
                    del playlist_tracks
                
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

100%|██████████| 1000/1000 [10:12<00:00,  1.63it/s] 


In [4]:
with open('artists.json', 'w') as decoder_file:
    json.dump(artist, decoder_file, indent=4)

with open('albums.json', 'w') as decoder_file:
    json.dump(album, decoder_file, indent=4)

with open('tracks.json', 'w') as decoder_file:
    json.dump(tracks, decoder_file, indent=4)

del artist, album, track

In [5]:
# Convert playlists data to a Pandas DataFrame
data = pd.DataFrame(playlists_data)

del playlists_data

# Save the DataFrame to a file
output_dir = r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes'
os.makedirs(output_dir, exist_ok=True)

# Save as CSV instead of Parquet
output_file = os.path.join(output_dir, "playlists_nn2.csv")
data.to_csv(output_file, index=False)


print(f"Processing completed. DataFrame saved to {output_file}.")

Processing completed. DataFrame saved to C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\playlists_nn2.csv.
